In [2]:
import numpy as np
import os,sys
sys.path.append('..')
from trainer import Trainer
from config import get_config
from utils import *
import pickle
import matplotlib.pyplot as plt
import math
from tqdm import trange
#from re_main import main
#from trainer import *

from pprint import pprint
from models import *

/home/slcf/.pyenv/versions/3.6.2/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [3]:
def norm_img(img):
    img = img/255.0
    return img
def denorm_img(img):
    return tf.clip_by_value(img*255.0,0,255)


In [4]:
def corrupt(x, noise_type='gaussian',noise_param=0.3):
    if noise_type=='corrupt':
        mask_dist = dist.Bernoulli(probs=1-noise_param,dtype=tf.float32)
        mask = mask_dist.sample(sample_shape=tf.shape(x))
        corrupted = tf.multiply(x,mask)    
    elif noise_type == 'pipSalt':
        noise_dist = dist.Categorial(probs=[noise_paaram/2.1-noise_param,noise_param/2])
        noise = tf.cast(noise_dist.sample(sample_shape = tf.shape(x))-1,dtype = tf.float32)
        corrupted = tf.clip_by_value(tf.add(x,noise),0.0,1.0)
    else:
        noise = tf.random_normal(shape=tf.shape(x),mean=0.0,stddev = noise_param)
        corrupted = tf.clip_by_value(tf.add(x,noise),0.0,1.0)
        
    return corrupted
        

In [5]:
def AE_dense(X=None,dims = [784,1024,1024],activation=tf.nn.elu,tied=False):
    shapeX = X.shape
    with tf.variable_scope("AE_dense") as vs:
        z = tf.reshape(X,[-1,np.prod(shapeX[1:])])
        for idx, n_output in enumerate(dims[1:]):
            z = slim.fully_connected(z,n_output,activation_fn = activation)
            z = slim.batch_norm(z)
            
        rx = z
        for idx, n_output in enumerate(dims[:-1][::-1]):
            if idx<len(dims)-2:
                rx = slim.fully_connected(rx,n_output,activation_fn = activation)
                rx = slim.batch_norm(rx)
            else:
                rx = slim.fully_connected(rx,n_output,activation_fn = tf.nn.sigmoid)
    variables = tf.contrib.framework.get_variables(vs)
    return rx, z, variables
    

In [6]:
### parameter specification ###
layers = [784,1024,1024]
lr_init = 0.1
lr_step = 100
lr_decay = 0.1
iterations = 10000
model_dir = '../logs/mnist_denseDAE'

In [7]:
tf.reset_default_graph()
mnist = tf.keras.datasets.mnist
(x_train,y_train),(x_test,y_test) = mnist.load_data()

In [8]:
X = tf.constant(x_train)
y = tf.constant(y_train)

In [9]:
print(X)
print(X.shape)

Tensor("Const:0", shape=(60000, 28, 28), dtype=uint8)
(60000, 28, 28)


In [10]:
image_batch, label_batch = tf.train.shuffle_batch(
[X,y],
batch_size = 64,
num_threads = 4,
capacity = 5000,
enqueue_many=True,
min_after_dequeue = 100,name = 'training_data')
image_batch = tf.to_float(image_batch)
label_batch = tf.to_float(label_batch)

In [11]:
#Reset default graph 하면 이전에 정의해 놓았던 tensor들이 모호해 지므로 적당한 위치에
#놓는 것이 매우 중요하다.

step_var = tf.Variable(0,name='step',trainable=False)
lr = tf.train.exponential_decay(lr_init, step_var, lr_step,lr_decay,staircase=True)


batch_X = image_batch
batch_X = norm_img(batch_X)
crrX = corrupt(image_batch)#defualt is gaussian
batch_crrX = denorm_img(crrX)


x = tf.reshape(batch_X,[-1,layers[0]])
rx,z, variables = AE_dense(crrX,layers,tf.nn.elu)
batch_rx = denorm_img(rx)
optimizer = tf.train.AdamOptimizer(lr)
loss = tf.reduce_mean(tf.abs(rx-x))
optim = optimizer.minimize(loss,var_list = variables)
summary_op = tf.summary.merge([
    tf.summary.scalar("loss",loss),
    tf.summary.scalar("lr",lr)
])

init_op = tf.global_variables_initializer()
saver = tf.train.Saver()
summary_writer = tf.summary.FileWriter(model_dir)
sv = tf.train.Supervisor(logdir = model_dir,
                        is_chief = True, saver = saver,
                        summary_op = summary_op,
                        summary_writer = summary_writer,
                        save_model_secs = 100,
                        global_step = step_var,
                        ready_for_local_init_op = None)
gpu_options = tf.GPUOptions(allow_growth = True)
sess_config = tf.ConfigProto(allow_soft_placement=True,
                            gpu_options = gpu_options)

In [12]:
with sv.prepare_or_wait_for_session(config=sess_config) as sess:
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    sess.run(init_op)
    start_step = sess.run(step_var)
    for itr in trange(start_step,iterations):
        fetch_dict={
            'optim':optim
        }
        if itr%100==0:
            fetch_dict.update({
                "summary": summary_op,
                "loss":loss,
                "lr":lr
            })
        result = sess.run(fetch_dict)
        #print("sess end")
        if itr%100==0:
            print("[{}/{}] Loss:{:.6f}".format(itr,iterations,result["loss"]))
        #if itr%1000==999:
        #    save_path = saver.save(sess,model_dir+"mnist_denseDAE",global_step = step_var)
    coord.request_stop()
    coord.join(threads)

INFO:tensorflow:Restoring parameters from ../logs/mnist_denseDAE/model.ckpt-0
INFO:tensorflow:Starting standard services.
INFO:tensorflow:Saving checkpoint to path ../logs/mnist_denseDAE/model.ckpt
INFO:tensorflow:Starting queue runners.
INFO:tensorflow:step/sec: 0


  0%|          | 0/10000 [00:00<?, ?it/s]

INFO:tensorflow:Recording summary at step 0.


  0%|          | 14/10000 [00:00<08:13, 20.23it/s] 

[0/10000] Loss:0.476829


...

[9800/10000] Loss:0.085149


 99%|█████████▉| 9943/10000 [00:34<00:00, 287.10it/s]

[9900/10000] Loss:0.083310


100%|██████████| 10000/10000 [00:34<00:00, 287.18it/s]

INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.CancelledError'>, Enqueue operation was cancelled
	 [[Node: training_data/random_shuffle_queue_EnqueueMany = QueueEnqueueManyV2[Tcomponents=[DT_UINT8, DT_UINT8], timeout_ms=-1, _device="/job:localhost/replica:0/task:0/device:CPU:0"](training_data/random_shuffle_queue, Const, Const_1)]]
